In [1]:
import pandas as pd
import sklearn
import scipy
import numpy as np
import matplotlib.pyplot as plt
from html_sanitizer import Sanitizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import random as rnd
from sklearn.metrics import f1_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from nltk.stem import WordNetLemmatizer
import bs4

In [2]:
data = pd.read_csv("Inf.csv")

In [3]:
y_train = np.array(data["target"])
groups_train = np.array(data["group"])
texts = np.array(data["information"])

In [4]:
len(groups_train)

11690

In [5]:
for i in range(len(texts)):
    if texts[i] is np.nan:
        texts[i] = ""
        

In [6]:
vect = TfidfVectorizer(min_df=7, max_df=0.4)
X_train = vect.fit_transform(texts)


In [79]:
# X_train1 = []
# for i in range(texts.shape[0]):
#     print(i)
#     cur_grp = groups_train[i]
#     words = set(texts[i].strip().split())
#     all_dist = []
#     #print(texts[groups_train == cur_grp])
#     for j in range(len(texts[groups_train == cur_grp])):
#         if i == j:
#             continue
        
        
        
#         words_j = set(texts[j].strip().split())
#         all_dist.append(len(words.intersection(words_j)))
#     X_train1.append(sorted(all_dist, reverse=True)[0:15]    )
# X_train1 = np.array(X_train1)    

In [80]:

neigh = 26
XTrain = []
for i in np.unique(groups_train):
    X_ = X_train[groups_train == i]
    
    distMart = sklearn.metrics.pairwise_distances(X_, metric="cosine")
    #print(distMart)
    meanDist = np.sort(distMart, axis=1)[:,1:neigh]
    lis = []
    #print(meanDist)
    for i in meanDist:
        XTrain.append(list(i))
        #print(i.shape)
        
    

    
#XTrain = np.array(XTrain)
    


In [81]:
XTrain = np.array(XTrain)

In [82]:
with open("DBSCAN_VECTOR.txt", "r") as f:
    lis = [int(i) for i in f.read().split()]
    print(len(lis))
    XTrain = np.append(XTrain[:,0:17], np.array(lis).reshape((-1,1)), axis=1)

11690


In [83]:
scl = StandardScaler()
scl.fit(XTrain)
XTrain = scl.transform(XTrain)

In [84]:
p = y_train.mean()

In [85]:
XTrain.shape

(11690, 18)

In [86]:
alpha = 0.000000000001
max_epoch = 500
C = 0.5
batch_size = 450
folds = 8
res = []
threeshold = 0.30
mean_roc_auc = 0
mean_f1 = 0

for train_index, test_index in GroupKFold(n_splits=folds).split(X_train, y_train, groups_train):
    #threeshold = 0.1
    xTrain = XTrain[train_index]
    yTrain = y_train[train_index]
    
    xTest = XTrain[test_index]
    yTest = y_train[test_index]
    
    clf = MySGDClassifier(batch_generator, C=C, max_epoch=max_epoch, 
                      model_type="log_reg",batch_size=batch_size)
    clf.fit(xTrain, yTrain)
    a1 = clf.predict(xTest)
    
    
    threshold = np.quantile(a1, 1-p)
    print("Roc ", roc_auc_score(yTest, a1))
    print(a1)
    mean_roc_auc += roc_auc_score(yTest, a1)
    
    a1 = (a1 > threeshold).astype(int)
    #print(np.unique(a1, return_counts=True))
    #print(np.unique(a1, return_counts=True)[1][0] / np.unique(a1, return_counts=True)[1][1])
    mean_f1 += f1_score(yTest, a1)
#     print(a2)
#     for i in range(40):
#         print("f1", f1_score(yTest, (a1 > threeshold).astype(int)), threeshold)
#         threeshold+=0.01

print(mean_roc_auc / folds)
print(mean_f1 / folds)

(1452, 18) (19,)
Roc  0.8436839937425307
[0.45717699 0.33562236 0.3690735  ... 0.10769141 0.11361731 0.12047619]
(1457, 18) (19,)
Roc  0.8674375702487416
[0.33106534 0.34152324 0.07227599 ... 0.75687192 0.69278013 0.59866855]
(1462, 18) (19,)
Roc  0.8433080030601583
[0.13322359 0.13119109 0.17410008 ... 0.22824141 0.1841898  0.22567161]
(1462, 18) (19,)
Roc  0.8981379291862485
[0.11887374 0.16224319 0.11887374 ... 0.57989782 0.73884728 0.33851585]
(1475, 18) (19,)
Roc  0.8769988993518406
[0.08227002 0.08715222 0.24372697 ... 0.73098729 0.58556957 0.8389528 ]
(1461, 18) (19,)
Roc  0.9075283927296184
[0.09244216 0.12896166 0.13082494 ... 0.17893192 0.1695244  0.54661704]
(1459, 18) (19,)
Roc  0.9235438403156717
[0.12313705 0.11169155 0.11996935 ... 0.18029006 0.12838655 0.18386575]
(1462, 18) (19,)
Roc  0.8580943330052383
[0.06161956 0.35912591 0.08323116 ... 0.72283761 0.69252705 0.6475501 ]
0.8773416202050061
0.701397545467939


In [87]:
#%%pycodestyle

def sigmoid(x):
    """
    Вычисляем значение сигмоида.
    X - выход линейной модели
    """
    
    sigm_value_x = 1. / (1 + np.exp(-x))
    return sigm_value_x


from sklearn.base import BaseEstimator, ClassifierMixin

class MySGDClassifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, batch_generator, C=1, alpha=0.01, 
                 max_epoch=10, model_type='lin_reg',
                batch_size=1):
        """
        batch_generator -- функция генератор, которой будем создавать батчи
        C - коэф. регуляризации
        alpha - скорость спуска
        max_epoch - максимальное количество эпох
        model_type - тип модели, lin_reg или log_reg
        """
        if model_type != 'lin_reg' and model_type != 'log_reg':
            raise TypeError
            
        self.batch_size = batch_size   
        self.C = C
        self.alpha = alpha
        self.max_epoch = max_epoch
        self.batch_generator = batch_generator
        self.errors_log = {'iter' : [], 'loss' : []}  
        self.model_type = model_type
        
    def calc_loss(self, X_batch, y_batch):#############
        """
        Считаем функцию потерь по батчу 
        X_batch - матрица объекты-признаки по батчу
        y_batch - вектор ответов по батчу
        Не забудте тип модели (линейная или логистическая регрессия)!
        """
        
        loss = 0
        if self.model_type == 'log_reg':
            p = sigmoid(np.dot(X_batch, self.weights))
            if p < 1e-16:
                p = 0.0001
            
            if 1 - p < 1e-16:
                p = 0.9999
                
            loss = np.dot(y_batch, np.log(p))
            loss += np.dot(1 - y_batch, np.log(1 - p))
            loss *= -1
            loss /= y_batch.shape[0]
        
        else:
            loss = ((y_batch - np.dot(X_batch, self.weights)) ** 2).sum()
            loss /= y_batch.shape[0]
            
        return loss 
    
    
    def calc_loss_grad(self, X_batch, y_batch):
        """
        Считаем  градиент функции потерь по батчу (то что Вы вывели в задании 1)
        X_batch - матрица объекты-признаки по батчу
        y_batch - вектор ответов по батчу
        Не забудте тип модели (линейная или логистическая регрессия)!
        """
        loss_grad = 0
        if self.model_type == 'lin_reg':
            loss_grad = np.dot(X_batch, self.weights) 
            loss_grad -= y_batch
            loss_grad = np.dot(X_batch.transpose(), loss_grad)
            loss_grad /= y_batch.shape[0]
            
            temp = self.weights[0]
            self.weights[0] = 0.
            loss_grad += self.C * self.weights
            self.weights[0] = temp
                
            loss_grad *= 2
            
        else:
            V = sigmoid(np.dot(X_batch, self.weights))
            loss_grad = V - y_batch
            loss_grad = np.dot(X_batch.transpose(), loss_grad)
            loss_grad /= y_batch.shape[0]
            
            temp = self.weights[0]
            self.weights[0] = 0.
            loss_grad += 2 * self.C * self.weights
            self.weights[0] = temp
         
        return loss_grad
    
    def update_weights(self, new_grad):####################
        """
        Обновляем вектор весов
        new_grad - градиент по батчу
        """
        self.weights -= new_grad * self.alpha
    
    def fit(self, X, y):
        '''
        Обучение модели
        X - матрица объекты-признаки
        y - вектор ответов
        '''
        
        # Нужно инициализровать случайно веса
        X = np.array(X)
        y = np.array(y)
        ones = np.array([1 for i in range(X.shape[0])]).reshape((-1,1))
        X = np.concatenate((ones, X), axis=1)
        
        self.weights = np.random.uniform(-10, 10, X.shape[1])
        for n in range(0, self.max_epoch):
            new_epoch_generator = self.batch_generator(X, y)
            iterator = 0
            for batch_num, new_batch in new_epoch_generator:
                X_batch = new_batch[0]
                
                y_batch = new_batch[1]
                batch_grad = self.calc_loss_grad(X_batch, y_batch)
                
                self.update_weights(batch_grad)
                batch_loss = self.calc_loss(X_batch, y_batch)
                
                self.errors_log['iter'].append(batch_num)
                self.errors_log['loss'].append(batch_loss)
                
                if iterator > X.shape[0] / self.batch_size:
                    break
                iterator+=1
                
                
        return self
        
    def predict(self, X):
        '''
        Предсказание класса
        X - матрица объекты-признаки
        Не забудте тип модели (линейная или логистическая регрессия)!
        '''
        print(X.shape, self.weights.shape)
        y_hat = 0
        if self.model_type == "lin_reg":
            y_hat = np.dot(X, self.weights[1:]) + self.weights[0]
        
        else:
            y_hat = sigmoid(np.dot(X, self.weights[1:]) + self.weights[0])
        # Желательно здесь использовать матричные операции между X и весами, например, numpy.dot 
        return y_hat

In [88]:
def batch_generator(X, y, shuffle=True, batch_size=1):
    """
    Гератор новых батчей для обучения
    X          - матрица объекты-признаки
    y_batch    - вектор ответов
    shuffle    - нужно ли случайно перемешивать выборку
    batch_size - размер батча ( 1 это SGD, > 1 mini-batch GD)
    Генерирует подвыборку для итерации спуска (X_batch, y_batch)
    """
    
    X_batch = ""
    y_batch = ""
    num = 0
    if shuffle == True:
        while True:
            
            ind = rnd.sample([i for i in range(X.shape[0])], batch_size)
            yield (num, (X[ind,:], y[ind]))
            num += 1
            
    else:
        i = 0
        n = X.shape[0]
        while True:
            ind = [t % n for t in range(i, i + batch_size)]
            i += batch_size
            yield (num, (X[ind,:], y[ind]))
            num += 1

In [89]:
p = y_train.mean()

In [90]:
p

0.28751069289991443

In [91]:
all_titles = pd.read_csv("docs_titles.tsv", sep="\t")
test_groups = pd.read_csv("test_groups.csv")
#table = test_groups.join(all_titles, lsuffix='_caller', rsuffix='_other', on="doc_id", how="inner")[["doc_id_caller","doc_id_other","pair_id", "group_id","title"]]
#table.shape

In [92]:
table = pd.merge(test_groups, all_titles, left_on='doc_id', right_on='doc_id',how="left")
table.shape

(16627, 4)

In [93]:
table.head()

,pair_id,group_id,doc_id,title
0,11691,130,6710,КАК ПРОПИСАТЬ АДМИНКУ В КС 1.6 СЕБЕ ИЛИ ДРУГУ ...
1,11692,130,4030,Скачать: SGL-RP доработка | Слив мода [MySQL] ...
2,11693,130,5561,Как прописать админку в кс 1.6 - Counter-Strik...
3,11694,130,4055,Как прописать простую админку в кс 1 6
4,11695,130,4247,Подбор админов для сервера по КОД_4 [Архив] -...


In [94]:
groups_test = np.array(table["group_id"])
texts = np.array(table["title"])
for i in range(len(texts)):
    if texts[i] is np.nan:
        texts[i] = ""
        

In [97]:
mystem = Mystem() 
rus_stopwords = stopwords.words("russian")
eng_stopwords = stopwords.words("english")
html_stopwords = ["span", "br", "a", "href", "img", "www", "com", "google", "ru", "html", "http", "https"]
stemming("Привет, медвед я в доме у  жен")

'привет  медведы   дом   жена\n'

In [96]:
def delete_symbols(text):
    res = ""
    text = text.lower()
    
    for i in text:
        t = i    
        if not(
                (t >= '0' and t <= '9') or 
                (t >= 'a' and t <= 'z') or
                (t >= 'а' and t <= 'я')
              ):
            t = ' '
            
        res += t
        
    return res 


def list2string(myList):  
    myStr = ""  
    
    for word in myList:
        if (word not in rus_stopwords) and (word not in eng_stopwords ) and (word not in html_stopwords):
            myStr += word
    
    return myStr

def stemming(text):
    text = delete_symbols(text)
    
    tokens = mystem.lemmatize(text)
    stri = list2string(tokens)
    #print(stri)
    return stri
    

In [98]:
new_texts = []
for i in texts:
    new_texts.append(stemming(i))

In [100]:
vect = TfidfVectorizer(min_df=5, max_df=0.5)
X_test = vect.fit_transform(new_texts)

In [101]:
pairs_id = []
groups_id = []

In [102]:

neigh = 26
XTest = []
for i in np.unique(groups_test):
    X_ = X_test[groups_test == i]
    
    distMart = sklearn.metrics.pairwise_distances(X_, metric="cosine")
    #print(distMart)
    meanDist = np.sort(distMart, axis=1)[:,1:neigh]
    
    for i in meanDist:
        XTest.append(list(i))
        #print(i.shape)
        
    

            
    

    
#XTrain = np.array(XTrain)
    


In [106]:
XTest = np.array(XTest)

In [107]:
with open("TEST_DBSCAN_VECTOR.txt", "r") as f:
    lis = [int(i) for i in f.read().split()]
    print(len(lis),XTest.shape)
    XTest = np.append(XTest[:,0:17], np.array(lis).reshape((-1,1)), axis=1)

16627 (16627, 25)


In [108]:
XTest = scl.transform(XTest)

In [109]:
alpha = 0.000000000001
max_epoch = 500
C = 0.5
batch_size = 450
folds = 8
res = []
threeshold = 0.3
mean_roc_auc = 0
mean_f1 = 0
res = []
r = []
for train_index, test_index in GroupKFold(n_splits=folds).split(XTrain, y_train, groups_train):
    #threeshold = 0.1
    xTrain = XTrain[train_index]
    yTrain = y_train[train_index]
    
    
    
    clf = MySGDClassifier(batch_generator, C=C, max_epoch=max_epoch, 
                      model_type="log_reg",batch_size=batch_size)
    clf.fit(xTrain, yTrain)
    a1 = clf.predict(XTest)
    threeshold = np.quantile(a1, 0.27)
    
    #print(a1)
    a1 = (a1 > threeshold).astype(int)
    #print(np.unique(a1, return_counts=True))
    #print(np.unique(a1, return_counts=True)[1][0] / np.unique(a1, return_counts=True)[1][1])
#     print(yTrain.shape, a1.shape)
    #print("Roc ", roc_auc_score(yTest, a1))
    res.append(a1)
    #mean_roc_auc += roc_auc_score(yTest, a1)
    #mean_f1 += f1_score(yTest, (a1 > threeshold).astype(int))

    

print(mean_roc_auc / folds)
print(mean_f1 / folds)

(16627, 18) (19,)
(16627, 18) (19,)
(16627, 18) (19,)
(16627, 18) (19,)
(16627, 18) (19,)
(16627, 18) (19,)
(16627, 18) (19,)
(16627, 18) (19,)
0.0
0.0


In [110]:
for i in range(folds):
    print(np.unique(res[i], return_counts=True))

(array([0, 1]), array([ 4490, 12137]))
(array([0, 1]), array([ 4490, 12137]))
(array([0, 1]), array([ 4490, 12137]))
(array([0, 1]), array([ 4490, 12137]))
(array([0, 1]), array([ 4490, 12137]))
(array([0, 1]), array([ 4490, 12137]))
(array([0, 1]), array([ 4490, 12137]))
(array([0, 1]), array([ 4490, 12137]))


In [111]:
res[0][res[0]!=0].shape

(12137,)

In [112]:
res1 = np.zeros((16627,))
for i in res:
    res1 += i
    
res1 = (res1 > 5).astype(int)
res = res1
with open("my_submission2.csv", "w") as f:
    print("pair_id,target", file=f)
    for i in range(len(res)):
        print(table["pair_id"].iloc[i], ",", res[i], file=f, sep="")

In [572]:
res[res!=1].shape

(11520,)

In [559]:
res[0][res[0]!=0].shape

(0,)

In [376]:
xTest

array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.33936409, 0.46239349, 0.47242415, ..., 1.        , 1.        ,
        1.        ],
       [0.66392517, 0.68379423, 0.69208193, ..., 1.        , 1.        ,
        1.        ],
       ...,
       [0.19342149, 0.20520456, 0.21564475, ..., 0.46177163, 0.47142318,
        0.47142318],
       [0.        , 0.22669694, 0.26326191, ..., 0.47526927, 0.49494703,
        0.51108142],
       [0.28213007, 0.28262285, 0.34750365, ..., 0.49420106, 0.50914478,
        0.50991972]])

In [478]:
neigh = 26
XTest = []
res = []
for i in np.unique(groups_train):
    y_ = y_train[groups_train == i]
    elem, cnt = np.unique(y_train[groups_train == i], return_counts=True)
    print(elem,cnt)
    if cnt[0] != 0:
        res.append(cnt[0]/cnt[1])
            
    

    
#XTrain = np.array(XTrain)

[0 1] [94  8]
[0 1] [80 14]
[0 1] [89  9]
[0 1] [81  7]
[0 1] [99  9]
[0 1] [25 46]
[0 1] [89  7]
[0 1] [65 26]
[0 1] [12 19]
[0 1] [84 13]
[0 1] [90  8]
[0 1] [59  6]
[0 1] [16 15]
[0 1] [74 18]
[0 1] [93  8]
[0 1] [51 43]
[0 1] [29 73]
[0 1] [74 15]
[0 1] [83  7]
[0 1] [90  5]
[0 1] [89 11]
[0 1] [12 75]
[0 1] [39 54]
[0 1] [52 50]
[0 1] [11 73]
[0 1] [ 5 74]
[0 1] [93  7]
[0 1] [85  6]
[0 1] [92  7]
[0 1] [95  9]
[0 1] [85 10]
[0 1] [96  9]
[0 1] [ 5 88]
[0 1] [44 54]
[0 1] [89 10]
[0 1] [29 74]
[0 1] [16 42]
[0 1] [75 29]
[0 1] [92  7]
[0 1] [92  6]
[0 1] [59  6]
[0 1] [29 74]
[0 1] [87  6]
[0 1] [80  8]
[0 1] [46 37]
[0 1] [90  7]
[0 1] [84 10]
[0 1] [33 22]
[0 1] [ 7 93]
[0 1] [ 6 82]
[0 1] [95  5]
[0 1] [25 74]
[0 1] [10 16]
[0 1] [92  8]
[0 1] [88  7]
[0 1] [84  8]
[0 1] [75 13]
[0 1] [92  6]
[0 1] [99  4]
[0 1] [11 70]
[0 1] [87  6]
[0 1] [100   5]
[0 1] [81  5]
[0 1] [86 10]
[0 1] [83  9]
[0 1] [86  5]
[0 1] [45 47]
[0 1] [92 12]
[0 1] [40 64]
[0 1] [95  7]
[0 1] [55 35]
[0 1

In [479]:
np.array(res).mean()

7.545438304536387